<a href="https://colab.research.google.com/github/dudeurv/SAM_MRI/blob/main/mri_classes_to_two_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install nilearn nibabel SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 5.3 MB/s eta 0:00:00


In [ ]:
# Part 1: Setup
import os
import zipfile
import SimpleITK as sitk
import numpy as np
import nibabel as nib
import gdown
import shutil


# Part 2: Data Preparation
# Download the dataset
!gdown https://drive.google.com/uc?id=1qTbg3oJlO_RXwsgjvpyzfFZw5K7KZjVm

# Unzip the dataset
with zipfile.ZipFile('endonasal_mri_corrected.zip', 'r') as zip_ref:
    zip_ref.extractall()

Downloading...
From: https://drive.google.com/uc?id=1qTbg3oJlO_RXwsgjvpyzfFZw5K7KZjVm
To: /content/endonasal_mri_corrected.zip
100% 358M/358M [00:05<00:00, 67.8MB/s]


In [ ]:
base_path = '/content/endonasal_mri_new'
output_folder = '/content/endonasal_mri_new'

os.makedirs(output_folder, exist_ok=True)

# Part 3: Processing Functions
def process_mri_with_itk(mri_folder):
    ica_path = os.path.join(mri_folder, f"{os.path.basename(mri_folder)}_ICAs.nii.gz")
    pituitary_tumour_path = os.path.join(mri_folder, f"{os.path.basename(mri_folder)}_PituitaryAndTumour.nii.gz")

    # Load images
    ica_img = sitk.ReadImage(ica_path)
    pituitary_tumour_img = sitk.ReadImage(pituitary_tumour_path)

    # Convert to arrays for processing
    ica_array = sitk.GetArrayFromImage(ica_img)
    pituitary_tumour_array = sitk.GetArrayFromImage(pituitary_tumour_img)

    # Reassign the class of ICA from 1 to 2 and combine ICAs with PituitaryAndTumour
    ica_array[ica_array == 1] = 2
    combined_array = ica_array + pituitary_tumour_array

    # Now explicitly reassign any third class as the second class if necessary
    combined_array[combined_array == 3] = 2

    # Convert array back to image and save
    combined_img = sitk.GetImageFromArray(combined_array)
    combined_img.CopyInformation(ica_img)
    output_path = os.path.join(mri_folder, f"{os.path.basename(mri_folder)}_mask.nii.gz")
    sitk.WriteImage(combined_img, output_path)

    # Remove old masks
    os.remove(ica_path)
    os.remove(pituitary_tumour_path)

    # Checking unique classes for debugging
    unique_classes = np.unique(combined_array)
    print(f"Unique classes in {os.path.basename(mri_folder)} combined image: {unique_classes}")

def process_anatomical_mask_with_itk(mask_path, mri_folder):
    # Adjust mask_path to include base_path
    mask_path = os.path.join(base_path, mask_path)

    # Load the mask image
    mask_img = sitk.ReadImage(mask_path)
    mask_array = sitk.GetArrayFromImage(mask_img)

    # Remove class 2 and reassign class 3 to 2
    mask_array[mask_array == 2] = 0
    mask_array[mask_array == 3] = 2

    # Save the modified mask
    modified_mask_img = sitk.GetImageFromArray(mask_array)
    modified_mask_img.CopyInformation(mask_img)
    output_path = os.path.join(mri_folder, "mri0066_mask.nii.gz")
    sitk.WriteImage(modified_mask_img, output_path)

    unique_classes = np.unique(mask_array)
    print(f"Unique classes in {os.path.basename(mri_folder)} combined image: {unique_classes}")

    # Remove old mask
    os.remove(mask_path)

    t1c_path = mask_path.replace('mask_reorderx', 't1c')
    t1c_img = sitk.ReadImage(t1c_path)
    t1c_path = t1c_path.replace('.nii', '.nii.gz')
    sitk.WriteImage(t1c_img, t1c_path)
    os.remove(t1c_path)

# Part 4: Execution - Go through the files in the Google Drive folder
for i in [153, 155, 167, 170, 169, 154, 164, 151, 152, 66]:
    mri_folder = f"mri00{i}" if i==66 else f"mri0{i}"
    full_mri_path = os.path.join(base_path, mri_folder)  # Adjusted to correct folder structure

    # Check if it's the special case of mri0010
    if i == 66:
        mask_path = os.path.join(full_mri_path, f'{mri_folder}_mask_reorderx.nii')  # Adjust for the correct file
        process_anatomical_mask_with_itk(mask_path, full_mri_path)
    else:
        # Process regular MRI folder
        process_mri_with_itk(full_mri_path)

# Part 5: Zip and Download Processed Images
zip_path = '/content/endonasal_mri_new.zip'
with zipfile.ZipFile(zip_path, 'w') as zip_f:
    for foldername, subfolders, filenames in os.walk(output_folder):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zip_f.write(file_path, os.path.relpath(file_path, output_folder))

print(f"Processed files are saved and zipped in {zip_path}")

Unique classes in mri0153 combined image: [0 1 2]
Unique classes in mri0155 combined image: [0 1 2]
Unique classes in mri0167 combined image: [0 1 2]
Unique classes in mri0170 combined image: [0 1 2]
Unique classes in mri0169 combined image: [0 1 2]
Unique classes in mri0154 combined image: [0 1 2]
Unique classes in mri0164 combined image: [0 1 2]
Unique classes in mri0151 combined image: [0 1 2]
Unique classes in mri0152 combined image: [0 1 2]
Unique classes in mri0066 combined image: [0. 1. 2.]
Processed files are saved and zipped in /content/endonasal_mri_new.zip


In [ ]:
# Delete any folders if needed
folder_path = input("Write a folder path, e.g. '/content/endonasal_mri_new': ")
shutil.rmtree(folder_path)
print(f"Successfully deleted folder: {folder_path}")

Write a folder path, e.g. '/content/endonasal_mri_new': /content/endonasal_mri_new_urvi
Successfully deleted folder: /content/endonasal_mri_new_urvi
